In [1]:
import pandas as pd
df = pd.read_csv("../Data/selected_data/processed_data.csv")

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217242 entries, 0 to 217241
Data columns (total 32 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   batter               217242 non-null  object 
 1   bowler               217242 non-null  object 
 2   non_striker          217242 non-null  object 
 3   runs_by_bat          217242 non-null  int64  
 4   extra_runs           217242 non-null  int64  
 5   total_runs_delivery  217242 non-null  int64  
 6   current_team_total   217242 non-null  int64  
 7   runs_remain          217242 non-null  float64
 8   batter_runs          217242 non-null  int64  
 9   balls_faced          217242 non-null  int64  
 10  wickets_fallen       217242 non-null  int64  
 11  extra_type           217242 non-null  object 
 12  delivery             217242 non-null  float64
 13  over                 217242 non-null  int64  
 14  wicket_type          217242 non-null  object 
 15  player_out       

In [12]:
# check winning_team column and if it is "no result" then remove the row
df = df[df["winning_team"] != "no result"]

In [14]:
# group data by match_id
grouped = df.groupby("match_id")

# get net run rate for each team
net_run_rate = {}
for match_id, group in grouped:
    team1 = group[group["innings"] == 1]
    team2 = group[group["innings"] == 2]

    team1_runs = team1["final_team_total"].iloc[0]
    team2_runs = team2["final_team_total"].iloc[0]
    
    team1_overs = team1["over"].iloc[-1] + (team1["delivery"].iloc[-1] / 6)
    team2_overs = team2["over"].iloc[-1] + (team2["delivery"].iloc[-1] / 6)
    

    # check if team1 is all out
    if team1["wickets_fallen"].iloc[-1] == 10:
        team1_overs = 20

    # check if team2 is all out
    if team2["wickets_fallen"].iloc[-1] == 10:
        team2_overs = 20

    team1_net_run_rate = (team1_runs / team1_overs) - (team2_runs / team2_overs)
    team2_net_run_rate = (team2_runs / team2_overs) - (team1_runs / team1_overs)

    # print(f"Match ID: {match_id}")
    # print(f"Team 1: {team1_net_run_rate}")
    # print(f"Team 2: {team2_net_run_rate}")
    # print()


    # introduce net_run_rate column for original dataframe df
    for index, row in group.iterrows():
        if row["innings"] == 1:
            df.at[index, "net_run_rate"] = team1_net_run_rate
        else:
            df.at[index, "net_run_rate"] = team2_net_run_rate

df.to_csv("../Data/selected_data/processed_data_NRR.csv", index=False)
